In [1]:
# this notebook contains code for the BIGANTR model
# modified to include variable abrasion

In [2]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# create a profile
dx = 500
x = np.arange(0, 50000, dx)
x_node = x + (dx/2)

# set sed thickness
Hstar = 0.5 # depth decay

In [4]:
# define constants and other parameters
K = 1e-6 # erodibility
beta_0 = 0.000005 # 0.00025 # sediment attrition # least abradable
beta_1 = 0.000005 # sediment attrition
beta_2 = 0.000005 # 0.00075 # sediment attrition # most abradable
zeta = 0.000005 # 0.001 # bedrock abrasion
r = 5. # runoff rate
U = 0.0001 # uplift or baselevel fall rate
I = 0.01 # intermittency
phi = 0.1 # porosity
D = 0.01 # grain size
gamma = 0.5 # fraction of plucked material that becomes coarse

kQs = 0.041 # sed transport coefficient
kxb = 25 # valley width coefficient
Pxb = 1./5. # valley width exponment
kb = 8.3e-8 # channel width coefficient

B_valley = kxb * (x_node**Pxb) # valley width
A = (1./3.) * (x**2)
Q = r * A

dt = 5 # timestep

In [5]:
def BIGANTR(x, dx, 
            K, beta_0, beta_1, beta_2, zeta, 
            gamma, D, phi, Hstar, 
            kQs, I, r, U, 
            B_valley, Q, 
            dt, num_steps = 1):
    
    # set up run duration and start counting time
    run_duration = dt * num_steps
    cum_time = 0.0
    
    # initialize arrays
    etab = np.zeros(len(x)) # bedrock elevation
    etab[:] = np.linspace(1, 0.1, len(x)) # start with slight bedrock slope

    H_0 = (Hstar/3) + np.zeros(len(x))
    H_1 = (Hstar/3) + np.zeros(len(x))
    H_2 = (Hstar/3) + np.zeros(len(x))
    H = H_0 + H_1 + H_2 # alluvium thickness

    eta = etab + H # topography
    
    Qs = np.zeros(len(x))
    
    f_0 = np.zeros(len(x))
    f_0[:] = (1./3.)
    Qs_0 = np.zeros(len(x))
    Eh_0 = np.zeros(len(x))
    
    f_1 = np.zeros(len(x))
    f_1[:] = (1./3.)
    Qs_1 = np.zeros(len(x))
    Eh_1 = np.zeros(len(x))
    
    f_2 = np.zeros(len(x))
    f_2[:] = (1./3.)
    Qs_2 = np.zeros(len(x))
    Eh_2 = np.zeros(len(x))
    
    Eb_p = np.zeros(len(x))
    Eb_a = np.zeros(len(x))
    Eb = np.zeros(len(x))
    Eh = np.zeros(len(x))
    E = np.zeros(len(x))
    
    ql = np.zeros(len(x)) # lateral sediment supply
    ql_0 = np.zeros(len(x)) # lateral sediment supply
    ql_1 = np.zeros(len(x)) # lateral sediment supply
    ql_2 = np.zeros(len(x)) # lateral sediment supply
    
    # set boundary conditions
    H[-1] = 0
    E[-1] = U
    Qs[0] = 0
    
    # track time
    while cum_time < run_duration:
        
        # drop baselevel
        eta[-1] -= U * dt
        
        # set boundary conditions
        etab[-1] = eta[-1]
        
        # calc slope
        S = -np.diff(eta)/dx
        
        # calc bed exposure
        alpha = np.exp(-H/Hstar)
        
        # calc sed transport
        Qs[1:] = kQs * I * Q[1:] * S**(7./6.) * (1 - alpha[:-1])
        
        # calc bedrock erosion
        Eb_p[:-1] = K * Q[1:] * S**(7./6.) * alpha[:-1] * I
        Eb_a[:-1] = zeta * Qs[1:] * alpha[:-1]
        Eb[:-1] = (Eb_p[:-1] + Eb_a[:-1]) / B_valley[:-1]
        
        # calc lateral flux
        ql_0[:-1] = (((Eb[:-1]/3) * gamma)/beta_0) * (1 - np.exp(-beta_0 * x[1:]/3))
        ql_1[:-1] = (((Eb[:-1]/3) * gamma)/beta_1) * (1 - np.exp(-beta_1 * x[1:]/3))
        ql_2[:-1] = (((Eb[:-1]/3) * gamma)/beta_2) * (1 - np.exp(-beta_2 * x[1:]/3))
        
        # calc Qs_0 (f_0 * Qs)
        Qs_0[1:] = f_0[1:] * Qs[1:]
        Qs_1[1:] = f_1[1:] * Qs[1:]
        Qs_2[1:] = f_2[1:] * Qs[1:]
        
        # calc atr_0 (Qs_0 * beta_0)
        atr_0 = Qs_0 * beta_0
        atr_1 = Qs_1 * beta_1
        atr_2 = Qs_2 * beta_2
        atr = atr_0 + atr_1 + atr_2
        
        # calc sediment erosion rate Eh_0 (use Qs_0, atr_0, Eb_p/3, and ql_0)
        Eh_0[:-1] = 1/((1 - phi) * B_valley[:-1]) * (np.diff(Qs_0)/dx + atr_0[1:] - ((Eb_p[:-1]/3)*gamma) - ql_0[:-1])
        Eh_1[:-1] = 1/((1 - phi) * B_valley[:-1]) * (np.diff(Qs_1)/dx + atr_1[1:] - ((Eb_p[:-1]/3)*gamma) - ql_1[:-1])
        Eh_2[:-1] = 1/((1 - phi) * B_valley[:-1]) * (np.diff(Qs_2)/dx + atr_2[1:] - ((Eb_p[:-1]/3)*gamma) - ql_2[:-1])
        Eh = Eh_0 + Eh_1 + Eh_2
        
        # calc H_0 (Eh_0 * dt)
        H_0[:-1] -= Eh_0[:-1] * dt
        H_1[:-1] -= Eh_1[:-1] * dt
        H_2[:-1] -= Eh_2[:-1] * dt
        
        # calc H (H_0 + H_1 + H_2)
        H = H_0 + H_1 + H_2
        
        # calc total erosion rate
        E[:-1] = Eb[:-1] + Eh[:-1]
        
        # update f_0 (H_0/H)
        f_0[1:] = H_0[:-1]/H[:-1] # 1 - (atr_0[1:] / atr[1:]) # first cell of f_0 should be constant at 1/3 ratio of total H
        f_1[1:] = H_1[:-1]/H[:-1] # 1 - (atr_1[1:] / atr[1:])
        f_2[1:] = H_2[:-1]/H[:-1] # 1 - (atr_2[1:] / atr[1:])
        
        # update topography
        etab[:-1] -= Eb[:-1] * dt
        H[:-1] -= Eh[:-1] * dt
        H[H<0] = 0
        eta[:-1] = etab[:-1] + H[:-1]
        
        # update elapsed time
        cum_time += dt
    
    # check total time (yrs)
    print(cum_time)
        
    return (dt, S, alpha, Qs, Eb_p, Eb_a, Eb, ql_0, ql_1, ql_2, f_0, f_1, f_2, 
            Qs_0, Qs_1, Qs_2, atr_0, atr_1, atr_2, Eh_0, Eh_1, Eh_2, Eh, E, etab, H, H_0, H_1, H_2, eta)

In [6]:
#now write a test
num_steps = 50000000 # 3000000

default_run = [x, dx, 
            K, beta_0, beta_1, beta_2, zeta, 
            gamma, D, phi, Hstar,
            kQs, I, r, U,
            B_valley, Q,
            dt, num_steps]

# and run it
(dt, S, alpha, Qs, Eb_p, Eb_a, Eb, ql_0, ql_1, ql_2, f_0, f_1, f_2, Qs_0, Qs_1, Qs_2, 
 atr_0, atr_1, atr_2, Eh_0, Eh_1, Eh_2, Eh, E, etab, H, H_0, H_1, H_2, eta) = BIGANTR(*default_run)

KeyboardInterrupt: 

In [ ]:
# check that we have reached steady state condition by printing out the bedrock erosion rate
Eb

In [ ]:
plt.plot(f_0, label = "most abrasion resistant")
plt.plot(f_1, label = "intermediate abrasion")
plt.plot(f_2, label = "most abradable")
plt.legend()

In [ ]:
f_0

In [ ]:
f_1

In [ ]:
f_2

In [ ]:
plt.plot(H_0)
plt.plot(H_1)
plt.plot(H_2)
plt.plot(H)

In [ ]:
H_0

In [ ]:
f_0+f_1+f_2

In [ ]:
plt.plot(Qs)
plt.plot(Qs_0)
plt.plot(Qs_1)
plt.plot(Qs_2)

In [ ]:
Qs_0

In [ ]:
# # save slopes
# slope_neg = np.savetxt('slope_neg.csv', S, delimiter = ',')

# # save total elevs
# eta_neg = np.savetxt('eta_neg.csv', eta, delimiter = ',')

# # save sed thickness
# H_neg = np.savetxt('H_neg.csv', H, delimiter = ',')

# # save Qs
# Qs_neg = np.savetxt('Qs_neg.csv', Qs, delimiter = ',')